In [ ]:
# как искать в xgb параметры - посмотреть в  google keeper (карпов)

In [ ]:
# подход Семина

In [ ]:
# auto ml

In [4]:
# модели

from sklearn.svm import SVC, SVR

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.neural_network import MLPClassifier, MLPRegressor

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier, \
                            RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor

from catboost import CatBoostClassifier, CatBoostRegressor

import xgboost as xgb 
from xgboost.sklearn import XGBClassifier, XGBRegressor  # conda install py-xgboost
import xgbfir

import lightgbm as lgb

import xgbfir

In [5]:
# sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize, scale

from sklearn.metrics import f1_score, recall_score, accuracy_score, roc_auc_score, precision_recall_fscore_support as score, \
                                    confusion_matrix, classification_report, mean_squared_error, mean_absolute_error

from sklearn.model_selection import train_test_split

In [7]:
# для выбора наилучшей модели
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval
# pip install future

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,   \
                                                cross_val_score, cross_val_predict, learning_curve, \
                                                KFold, StratifiedKFold, TimeSeriesSplit
# from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score, make_scorer
kappa_scorer = make_scorer(cohen_kappa_score)

In [9]:
# снижение размерности
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [8]:
import warnings
warnings.filterwarnings("ignore")

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [10]:
import os

In [ ]:
# посмотреть как предсказания завият от random state

# ensembling prediction
print('Starting the loop...')
num_ensembles = 6
y_test_pred = 0
for i in tqdm(range(num_ensambles)): # tqdm - for visualization
	model = CatBoostClassifier(random_seed=i+42, **params ) # params = depth=6, iterations = 800, ...
	fit_model =  model.fit(train_data)
	y_test_pred += fit_model.predict_proba(test_data)[:,1]
y_test_pred /= num_ensambles
gc.collect()

In [ ]:
# график предсказний

In [ ]:
# feature importance
def boosting_fi(alg, X, n_feat=30):

    feat_imp_df = pd.DataFrame({"Feature": X.columns, "Score": alg.feature_importances_})
    feat_imp_df = feat_imp_df.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

    plt.rcParams["figure.figsize"] = (20, n_feat//2)
    
    ax = feat_imp_df[0:n_feat].plot('Feature', 'Score', kind='barh',color=list(plt.rcParams['axes.prop_cycle'])[0]['color'])
    ax.invert_yaxis()
    ax.set_title("Feature Importance Ranking", fontsize = 14)
    plt.show()
    return feat_imp_df

# feat_imp_df = boosting_fi(alg, X_train_full, n_feat=30)

In [ ]:
# cumulative feature importance
def cum_fi(feat_imp_df, thr=0.95):
	feat_imp_df['importance_normalized'] = feat_imp_df['Score'] / feat_imp_df['Score'].sum()
	feat_imp_df['cumulative_importance'] = np.cumsum(feat_imp_df['importance_normalized'])

	cum_imp = list(feat_imp_df.cumulative_importance)
	feature_list = list(feat_imp_df["Feature"])
	x_values = list(range(feat_imp_df.shape[0]))

	fi_threshold = thr

	top_feat = feat_imp_df[feat_imp_df.cumulative_importance < fi_threshold]["Feature"]
	print('%d features needed for %d%% of cumulative importance' % (len(top_feat), fi_threshold*100))

	plt.figure(figsize = (16, 8))
	plt.rcParams['font.size'] = 13
	plt.plot(x_values, cum_imp, 'g-')

	# Draw line at thr% of importance retained
	plt.hlines(y = fi_threshold, xmin=0, xmax=len(feature_list), color = 'r', linestyles = 'dashed')

	plt.vlines(x = len(top_feat), ymin=0, ymax=1, color = 'black')

	# Format x ticks and labels
	plt.xticks(x_values, feature_list, rotation = 'vertical')
	# Axis labels and title
	plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

	return feat_imp_df, top_feat.values

# feat_df, top_feat = cum_fi(feat_imp_df, thr=0.95)

In [ ]:
# optimal lr/tree number 
n_estimators = np.arange(100,550,50)
learning_rate = np.geomspace(0.01, 1.0, 10)
param_grid = dict(learning_rate = learning_rate, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="recall", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results["params"]
for mean, stdev in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# делить модель по регионам 

In [ ]:
K = 10
cv = StratifiedKFold(n_splits=K, random_state=42, shuffle=True)

y_valid_pred = 0 * y_train_full
y_test_pred = 0

for i, (train_index, test_index) in enumerate(cv.split(X_train_full, y_train_full)):
    cb_init = cb.CatBoostClassifier(**init_params)
    # Create data for this fold
    y_train, y_valid = y_train_full.iloc[train_index], y_train_full.iloc[test_index]
    X_train, X_valid = X_train_full.iloc[train_index,:], X_train_full.iloc[test_index,:]
    print( "\nFold ", i+1)
    
    model_fit = cb_init.fit(X_train, y_train, 
                           cat_features=cat_feat_list,
                           eval_set=Pool(X_valid, y_valid, cat_features=cat_feat_list),
                           use_best_model=True,  
                           plot=True,
                           early_stopping_rounds=30,
                )
    
    print( " N trees = ", cb_init.tree_count_ )
    
#     cb_init.set_params(iterations=cb_init.tree_count_) # не используем, т.к. use_best_model=True
    
    # Generate validation predictions for this fold
    pred = model_fit.predict(X_valid)
    print( " AUC on cv fold = ", np.round(roc_auc_score(y_valid, pred),4)) # повторяет график (пик)
    y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    y_test_pred += model_fit.predict_proba(X_test)[:,1]
    
y_test_pred /= K  # Average test set predictions

print( "\nAUC for full training set: " , roc_auc_score(y_train_full, y_valid_pred))

In [11]:
# class RmseMetric(object):
#     def get_final_error(self, error, weight):
#         return np.sqrt(error / (weight + 1e-38))

#     def is_max_optimal(self):
#         return False

#     def evaluate(self, approxes, target, weight):
#         assert len(approxes) == 1
#         assert len(target) == len(approxes[0])

#         approx = approxes[0]

#         error_sum = 0.0
#         weight_sum = 0.0

#         for i in range(len(approx)):
#             w = 1.0 if weight is None else weight[i]
#             weight_sum += w
#             error_sum += w * ((approx[i] - target[i])**2)

#         return error_sum, weight_sum

In [14]:
class RmseMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            
            w = 1.0 if weight is None else weight[i]
            
            weight_sum += w
            
            dev = (target[i] - approx[i]) / target[i]
            
            const = 1.1
            
            if dev < -0.6:
                hit = 9 * const
            elif (dev >= -0.6) and (dev < -0.15):
                hit = const * (1 + dev/0.15)**2
            elif (dev >= -0.15) and (dev < 0.15):
                hit = 0
            elif (dev >= 0.15) and (dev < 0.6):
                hit = (dev/0.15 - 1)**2
            else:
                hit = 9
                
            error_sum += w * hit

        return error_sum, weight_sum